In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sys, os
from scipy.stats import poisson,skellam
from scipy.optimize import minimize

sys.path.insert(0, '/home/igormago/projects/DeepFootball')

from conf.configparser import conf
datasets_path = conf['path']['datasets']

plt.style.use('ggplot')

import statsmodels.api as sm
import statsmodels.formula.api as smf

### Reading Data

In [6]:
df_file = os.path.join(datasets_path,'be_matches','Brazil B.csv')
df = pd.read_csv(df_file, low_memory=False)
df['dt'] = pd.to_datetime(df['dt'],  format='%Y-%m-%d')

In [7]:
dtrain = df[(df.ss_yi >= 2013) & (df.ss_yi <= 2016)]
dtest = df[(df.ss_yi >= 2017) & (df.ss_yi <= 2018)]

In [5]:
dtrain_total = dtrain

for year in [2017,2018]:
    dround_pred = dtest[(dtest.ss_yi == year) & (dtest.rg < 5)]
    dtrain_total = pd.concat([dtrain_total,dround_pred], sort=True)
    
    for nround in range (5,39):
        dround_pred = dtest[(dtest.ss_yi == year) & (dtest.rg == nround)]
                
        dtrain_total['days_dif'] = (max(dtrain_total['dt']) - dtrain_total['dt']).dt.days
        dtrain_trans = pd.concat([dtrain_total[['th','ta','gh']].assign(home=1).rename(columns={'gh':'goals'}), 
                             dtrain_total[['ta','th','ga']].assign(home=0).rename(columns={'ga':'goals'})], sort=True)
        
        model = smf.glm(formula="goals ~ home + th + ta", data=dtrain_trans, 
                        family=sm.families.Poisson()).fit()
        
        for m in dround_pred.iterrows():
            th, ta = m[1]['th'], m[1]['ta']
            pyes, pno = simulate_match(model, th, ta, 10)
            dtest.loc[m[0],'pyes'] = pyes
            dtest.loc[m[0],'pno'] = pno
            
        dround_pred.loc[:,'pyes'] = lyes
        dround_pred.loc[:,'no'] = lno
            
        dtrain_total = pd.concat([dtrain_total,dround_pred], sort=True)
            

NameError: name 'simulate_match' is not defined